In [2]:
# import libraries
import boto3, re, sys, math, json, os, sagemaker, urllib.request
from sagemaker import get_execution_role

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import Image
from IPython.display import display
import time
from time import gmtime, strftime
#from sagemaker.predictor import csv_serializer

import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Dropout
from sagemaker.tensorflow import TensorFlow

from sagemaker.tensorflow import TensorFlowProcessor
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.transformer import Transformer
from sagemaker.image_uris import retrieve

from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, mean_absolute_percentage_error

import os
import keras

In [3]:
# Define IAM role
role = get_execution_role()
#pipeline_session = PipelineSession()
session = sagemaker.Session()
my_region = boto3.session.Session().region_name # set the region of the instance
aws_account_id = boto3.client('sts').get_caller_identity().get('Account')
print("RoleArn: {}".format(role))

RoleArn: arn:aws:iam::728939754004:role/service-role/AmazonSageMaker-ExecutionRole-20230821T133341


In [4]:
bucket_name = 'justin-automation-output'
prefix = 'outputs'

#s3 = boto3.resource('s3')

In [ ]:
!aws s3 cp --recursive s3://arbit-algo/sagemaker/algo-v1/input/model/ model/

### Preprocessing

Instructions/example for preprocessing file & training: https://github.com/aws/amazon-sagemaker-examples/blob/main/sagemaker_processing/scikit_learn_data_processing_and_model_evaluation/scikit_learn_data_processing_and_model_evaluation.ipynb

In [6]:
# change as needed
input_data = "s3://arbit-algo/sagemaker/algo-v1/processing/input/input.csv"
input_sku = "s3://arbit-algo/sagemaker/algo-v1/processing/input/input_sku.csv"

In [7]:
# documentation: https://sagemaker.readthedocs.io/en/stable/api/training/processing.html#sagemaker.processing.Processor
preprocessing_instance = 'ml.m5.2xlarge'
source_dir = 'model/'
#output_path = 's3://{}/{}/processing/output'.format(bucket_name, prefix)

#Initialize the TensorFlowProcessor
tp = TensorFlowProcessor(
    framework_version='2.3',
    role=get_execution_role(),
    instance_type=preprocessing_instance,
    instance_count=1,
    base_job_name='frameworkprocessor-TF',
    py_version='py37', 
    sagemaker_session=session#pipeline_session
    #output_path=output_path
)

In [9]:
#Run the processing job
# TODO: see how s3 source & destination impacts job
tp.run(
    code='preprocess.py',
    source_dir=source_dir,
    inputs=[
        ProcessingInput(source=input_data, destination="/opt/ml/processing/input"),
    ],
    outputs=[
        ProcessingOutput(output_name="train_data", source="/opt/ml/processing/train", destination='s3://justin-automation-output/outputs/processing/output/train'),
        ProcessingOutput(output_name="test_data", source="/opt/ml/processing/test", destination='s3://justin-automation-output/outputs/processing/output/test'),
    ],
    arguments=["--train-test-split-ratio", "0.2", "--model_type", "sneaker"]
)




Job Name:  frameworkprocessor-TF-2023-11-22-20-02-16-250
Inputs:  [{'InputName': 'input-1', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://arbit-algo/sagemaker/algo-v1/processing/input/input.csv', 'LocalPath': '/opt/ml/processing/input', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'code', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-728939754004/frameworkprocessor-TF-2023-11-22-20-02-16-250/source/sourcedir.tar.gz', 'LocalPath': '/opt/ml/processing/input/code/', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'entrypoint', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-728939754004/frameworkprocessor-TF-2023-11-22-20-02-16-250/source/runproc.sh', 'LocalPath': '/opt/ml/processing/input/entrypoint', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistr

### Training & Evaluation

#### More than 20 Sales

In [8]:
X_train_preprocessed = pd.read_csv('s3://arbit-algo/sagemaker/algo-v1/processing/output/train/X_train.csv')#.iloc[:,1:]
y_train = pd.read_csv('s3://arbit-algo/sagemaker/algo-v1/processing/output/train/y_train.csv')#.iloc[:,1:]

X_test_preprocessed = pd.read_csv('s3://arbit-algo/sagemaker/algo-v1/processing/output/test/X_test.csv')#.iloc[:,1:]
y_test = pd.read_csv('s3://arbit-algo/sagemaker/algo-v1/processing/output/test/y_test.csv')#.iloc[:,1:]

In [9]:
print('dim of X_train, model 1: ', X_train_preprocessed.shape)

dim of X_train, model 1:  (8652, 72)


In [10]:
## save new X_train, X_test, y_train, y_test from AWS locally, concat and save as training & validation csv

model_name = '1'

training = pd.concat([X_train_preprocessed, y_train], axis=1)
validation = pd.concat([X_test_preprocessed, y_test], axis=1)

print(f'Training Shape: {training.shape}')

training.to_csv(f'training_{model_name}.csv') #s3://arbit-algo/sagemaker/algo-v1/train/
validation.to_csv(f'validation_{model_name}.csv') #s3://arbit-algo/sagemaker/algo-v1/val/

# Save to S3 as training / validation

boto3.Session().resource('s3').Bucket(bucket_name).\
Object(os.path.join(prefix, f'train/training_{model_name}.csv')).upload_file(f'training_{model_name}.csv')
s3_input_train = sagemaker.inputs.TrainingInput(s3_data='s3://{}/{}/train'.format(bucket_name, prefix), 
                                                content_type='csv')

boto3.Session().resource('s3').Bucket(bucket_name).\
Object(os.path.join(prefix, f'val/validation_{model_name}.csv')).upload_file(f'validation_{model_name}.csv')
s3_input_val = sagemaker.inputs.TrainingInput(s3_data='s3://{}/{}/val'.format(bucket_name, prefix), 
                                              content_type='csv')

Training Shape: (8652, 73)


In [11]:
training_instance = 'ml.m5.2xlarge'
training_container = sagemaker.image_uris.retrieve(framework='tensorflow', region=my_region, version='2.3',
                                                   image_scope='training', instance_type=training_instance)

print("Success - the MySageMakerInstance is in the " + my_region + 
      " region. You will use the " + training_container + " container for your SageMaker endpoint.")

# Store model locally. A S3 URI would work too.
output_path = 's3://{}/{}/output/models/1/'.format(bucket_name, prefix)
source_dir = 'model/'
#training_instance = 'ml.m5.large'


tf_estimator = TensorFlow(entry_point='algo1.py',
                          role=role,
                          instance_count=1,
                          max_concurrent_transforms=100,
                          instance_type=training_instance,   
                          framework_version='2.2',  # try 2.2 "1.15
                          py_version='py37',
                          hyperparameters={'epochs': 40, 'batch_size': 32, 'model_name': '1'},
                          output_path=output_path, 
                          source_dir=source_dir, 
                          sagemaker_session=session, #pipeline_session
                          batch_strategy="SingleRecord"
                          # parameter 
                         )

# fit tensorflow estimator on training data
tf_estimator.fit({
    'train': 's3://{}/{}/train'.format(bucket_name, prefix),
    'test': 's3://{}/{}/val'.format(bucket_name, prefix),
    #'model_name': '/1'
})

# # inference, serving not working
# serving_instance = 'ml.m5.large'
# serving_container = sagemaker.image_uris.retrieve(framework='tensorflow', region=my_region, version='2.3',
#                                                   image_scope='inference', instance_type=serving_instance)

Success - the MySageMakerInstance is in the us-east-1 region. You will use the 763104351884.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training:2.3-cpu-py37 container for your SageMaker endpoint.
2023-12-13 17:10:23 Starting - Starting the training job...
2023-12-13 17:10:38 Starting - Preparing the instances for trainingProfilerReport-1702487409: InProgress
.........
2023-12-13 17:12:10 Downloading - Downloading input data...
2023-12-13 17:12:50 Training - Training image download completed. Training in progress...2023-12-13 17:13:01.168035: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:425] Initializing the SageMaker Profiler.
2023-12-13 17:13:01.168200: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:106] SageMaker Profiler is not enabled. The timeline writer thread will not be started, future recorded events will be dropped.
2023-12-13 17:13:01.189923: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:425] Initializing the SageMaker Profiler.
2023-12

#### More than 5 Sales

In [12]:
X_train_preprocessed = pd.read_csv('s3://arbit-algo/sagemaker/algo-v1/processing/output/train/X_train2.csv')#.iloc[:,1:]
y_train = pd.read_csv('s3://arbit-algo/sagemaker/algo-v1/processing/output/train/y_train2.csv')#.iloc[:,1:]

X_test_preprocessed = pd.read_csv('s3://arbit-algo/sagemaker/algo-v1/processing/output/test/X_test2.csv')#.iloc[:,1:]
y_test = pd.read_csv('s3://arbit-algo/sagemaker/algo-v1/processing/output/test/y_test2.csv')#.iloc[:,1:]

In [13]:
print('dim of X_train, model 2: ', X_train_preprocessed.shape)

dim of X_train, model 2:  (29052, 81)


In [14]:
## save new X_train, X_test, y_train, y_test from AWS locally, concat and save as training & validation csv

model_name = '2'

training = pd.concat([X_train_preprocessed, y_train], axis=1)
validation = pd.concat([X_test_preprocessed, y_test], axis=1)

print(f'Training Shape: {training.shape}')

training.to_csv(f'training_{model_name}.csv') # s3://arbit-algo/sagemaker/algo-v1/train/
validation.to_csv(f'validation_{model_name}.csv') # s3://arbit-algo/sagemaker/algo-v1/val/

# Save to S3 as training / validation

boto3.Session().resource('s3').Bucket(bucket_name).\
Object(os.path.join(prefix, f'train/training_{model_name}.csv')).upload_file(f'training_{model_name}.csv')
s3_input_train = sagemaker.inputs.TrainingInput(s3_data='s3://{}/{}/train'.format(bucket_name, prefix), 
                                                content_type='csv')

boto3.Session().resource('s3').Bucket(bucket_name).\
Object(os.path.join(prefix, f'val/validation_{model_name}.csv')).upload_file(f'validation_{model_name}.csv')
s3_input_val = sagemaker.inputs.TrainingInput(s3_data='s3://{}/{}/val'.format(bucket_name, prefix), 
                                              content_type='csv')

Training Shape: (29052, 82)


In [15]:
training_instance = 'ml.m5.2xlarge'
training_container = sagemaker.image_uris.retrieve(framework='tensorflow', region=my_region, version='2.3',
                                                   image_scope='training', instance_type=training_instance)

print("Success - the MySageMakerInstance is in the " + my_region + 
      " region. You will use the " + training_container + " container for your SageMaker endpoint.")

# Store model locally. A S3 URI would work too.
output_path = 's3://{}/{}/output/models/2/'.format(bucket_name, prefix)
source_dir = 'model/'
#training_instance = 'ml.m5.large'


tf_estimator = TensorFlow(entry_point='algo1.py',
                          role=role,
                          instance_count=1, 
                          instance_type=training_instance,   # Train on the local CPU ('local_gpu' if it has a GPU)
                          framework_version='2.2',  # try 2.2 "1.15
                          py_version='py37',
                          hyperparameters={'epochs': 40, 'batch_size': 32, 'model_name': '2'},
                          output_path=output_path, 
                          source_dir=source_dir, 
                          sagemaker_session=session#pipeline_session
                          # parameter 
                         )

# fit tensorflow estimator on training data
tf_estimator.fit({
    'train': 's3://{}/{}/train'.format(bucket_name, prefix),
    'test': 's3://{}/{}/val'.format(bucket_name, prefix),
    #'model_name': '/2'
})

# # inference, serving not working
# serving_instance = 'ml.m5.large'
# serving_container = sagemaker.image_uris.retrieve(framework='tensorflow', region=my_region, version='2.3',
#                                                   image_scope='inference', instance_type=serving_instance)

# endpoint_name = 'model2'
# predictor = tf_estimator.deploy(initial_instance_count=1, 
#                                 instance_type=serving_instance, 
#                                 model_name='1', endpoint_name=endpoint_name)

Success - the MySageMakerInstance is in the us-east-1 region. You will use the 763104351884.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training:2.3-cpu-py37 container for your SageMaker endpoint.
2023-12-13 17:14:57 Starting - Starting the training job...
2023-12-13 17:15:22 Starting - Preparing the instances for trainingProfilerReport-1702487683: InProgress
.........
2023-12-13 17:16:47 Downloading - Downloading input data...
2023-12-13 17:17:27 Downloading - Downloading the training image...
2023-12-13 17:17:43 Training - Training image download completed. Training in progress..2023-12-13 17:17:54.551827: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:425] Initializing the SageMaker Profiler.
2023-12-13 17:17:54.551975: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:106] SageMaker Profiler is not enabled. The timeline writer thread will not be started, future recorded events will be dropped.
2023-12-13 17:17:54.573250: W tensorflow/core/profiler/internal/smpro

#### SKU - More than 20

In [11]:
## SKU
#Run the processing job
# TODO: see how s3 source & destination impacts job
tp.run(
    code='preprocess.py',
    source_dir=source_dir,
    inputs=[
        #ProcessingInput(source=input_data, destination="/opt/ml/processing/input"),
        ProcessingInput(source=input_sku, destination="/opt/ml/processing/input_sku"),

    ],
    outputs=[
        ProcessingOutput(output_name="train_data", source="/opt/ml/processing/train", destination='s3://justin-automation-output/outputs/processing/output/train'),
        ProcessingOutput(output_name="test_data", source="/opt/ml/processing/test", destination='s3://justin-automation-output/outputs/processing/output/test'),
        # ProcessingOutput(
        #     output_name='output',
        #     source='/opt/ml/processing/output',
        #     destination='s3://arbit-algo/sagemaker/algo-v1/processing/output'
        # )
    ],
    arguments=["--train-test-split-ratio", "0.2", "--model_type", "sku"]
)


Job Name:  frameworkprocessor-TF-2023-12-13-15-31-35-353
Inputs:  [{'InputName': 'input-1', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://arbit-algo/sagemaker/algo-v1/processing/input/input_sku.csv', 'LocalPath': '/opt/ml/processing/input_sku', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'code', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-728939754004/frameworkprocessor-TF-2023-12-13-15-31-35-353/source/sourcedir.tar.gz', 'LocalPath': '/opt/ml/processing/input/code/', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'entrypoint', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-728939754004/frameworkprocessor-TF-2023-12-13-15-31-35-353/source/runproc.sh', 'LocalPath': '/opt/ml/processing/input/entrypoint', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3D

In [12]:
X_train_preprocessed = pd.read_csv('s3://arbit-algo/sagemaker/algo-v1/processing/output/train/X_train_sku.csv').iloc[:,1:]
y_train = pd.read_csv('s3://arbit-algo/sagemaker/algo-v1/processing/output/train/y_train_sku.csv')['SOLD_PRICE']#.iloc[:,1:]

X_test_preprocessed = pd.read_csv('s3://arbit-algo/sagemaker/algo-v1/processing/output/test/X_test_sku.csv').iloc[:,1:]
y_test = pd.read_csv('s3://arbit-algo/sagemaker/algo-v1/processing/output/test/y_test_sku.csv')['SOLD_PRICE']#.iloc[:,1:]

In [13]:
print('dim of X_train, model 3: ', X_train_preprocessed.shape)

dim of X_train, model 3:  (6419, 84)


In [14]:
## save new X_train, X_test, y_train, y_test from AWS locally, concat and save as training & validation csv

model_name = '3'

training = pd.concat([X_train_preprocessed, y_train], axis=1)
validation = pd.concat([X_test_preprocessed, y_test], axis=1)

print(f'Training Shape: {training.shape}')

training.to_csv(f'training_{model_name}.csv')
validation.to_csv(f'validation_{model_name}.csv')

# Save to S3 as training / validation

boto3.Session().resource('s3').Bucket(bucket_name).\
Object(os.path.join(prefix, f'train/training_{model_name}.csv')).upload_file(f'training_{model_name}.csv')
s3_input_train = sagemaker.inputs.TrainingInput(s3_data='s3://{}/{}/train'.format(bucket_name, prefix), 
                                                content_type='csv')

boto3.Session().resource('s3').Bucket(bucket_name).\
Object(os.path.join(prefix, f'val/validation_{model_name}.csv')).upload_file(f'validation_{model_name}.csv')
s3_input_val = sagemaker.inputs.TrainingInput(s3_data='s3://{}/{}/val'.format(bucket_name, prefix), 
                                              content_type='csv')

Training Shape: (6419, 85)


In [15]:
training_instance = 'ml.m5.2xlarge'
training_container = sagemaker.image_uris.retrieve(framework='tensorflow', region=my_region, version='2.3',
                                                   image_scope='training', instance_type=training_instance)

print("Success - the MySageMakerInstance is in the " + my_region + 
      " region. You will use the " + training_container + " container for your SageMaker endpoint.")

# Store model locally. A S3 URI would work too.
output_path = 's3://{}/{}/output/models/3/'.format(bucket_name, prefix)
source_dir = 'model/'



tf_estimator = TensorFlow(entry_point='algo1.py',
                          role=role,
                          instance_count=1, 
                          instance_type=training_instance,   # Train on the local CPU ('local_gpu' if it has a GPU)
                          framework_version='2.2',  # try 2.2 "1.15
                          py_version='py37',
                          hyperparameters={'epochs': 40, 'batch_size': 32, 'model_name': '3'},
                          output_path=output_path, 
                          source_dir=source_dir, 
                          sagemaker_session=session#pipeline_session
                          # parameter 
                         )

# fit tensorflow estimator on training data
tf_estimator.fit({
    'train': 's3://{}/{}/train'.format(bucket_name, prefix),
    'test': 's3://{}/{}/val'.format(bucket_name, prefix),
    #'model_name': '/2'
})


Success - the MySageMakerInstance is in the us-east-1 region. You will use the 763104351884.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training:2.3-cpu-py37 container for your SageMaker endpoint.
2023-12-12 16:38:32 Starting - Starting the training job...
2023-12-12 16:38:58 Starting - Preparing the instances for trainingProfilerReport-1702399098: InProgress
.........
2023-12-12 16:40:19 Downloading - Downloading input data...
2023-12-12 16:40:59 Training - Downloading the training image...
2023-12-12 16:41:19 Training - Training image download completed. Training in progress.2023-12-12 16:41:19.956703: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:425] Initializing the SageMaker Profiler.
2023-12-12 16:41:19.956845: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:106] SageMaker Profiler is not enabled. The timeline writer thread will not be started, future recorded events will be dropped.
2023-12-12 16:41:19.978163: W tensorflow/core/profiler/internal/smprofile

#### SKU - More than 1 sale

In [16]:
X_train_preprocessed = pd.read_csv('s3://arbit-algo/sagemaker/algo-v1/processing/output/train/X_train_sku2.csv').iloc[:,1:]
y_train = pd.read_csv('s3://arbit-algo/sagemaker/algo-v1/processing/output/train/y_train_sku2.csv')['SOLD_PRICE']#.iloc[:,1:]

X_test_preprocessed = pd.read_csv('s3://arbit-algo/sagemaker/algo-v1/processing/output/test/X_test_sku2.csv').iloc[:,1:]
y_test = pd.read_csv('s3://arbit-algo/sagemaker/algo-v1/processing/output/test/y_test_sku2.csv')['SOLD_PRICE']#.iloc[:,1:]

In [17]:
print('dim of X_train, model 4: ', X_train_preprocessed.shape)

dim of X_train, model 4:  (6891, 82)


In [18]:
## save new X_train, X_test, y_train, y_test from AWS locally, concat and save as training & validation csv

model_name = '4'

training = pd.concat([X_train_preprocessed, y_train], axis=1)
validation = pd.concat([X_test_preprocessed, y_test], axis=1)

print(f'Training Shape: {training.shape}')

training.to_csv(f'training_{model_name}.csv')
validation.to_csv(f'validation_{model_name}.csv')

# Save to S3 as training / validation

boto3.Session().resource('s3').Bucket(bucket_name).\
Object(os.path.join(prefix, f'train/training_{model_name}.csv')).upload_file(f'training_{model_name}.csv')
s3_input_train = sagemaker.inputs.TrainingInput(s3_data='s3://{}/{}/train'.format(bucket_name, prefix), 
                                                content_type='csv')

boto3.Session().resource('s3').Bucket(bucket_name).\
Object(os.path.join(prefix, f'val/validation_{model_name}.csv')).upload_file(f'validation_{model_name}.csv')
s3_input_val = sagemaker.inputs.TrainingInput(s3_data='s3://{}/{}/val'.format(bucket_name, prefix), 
                                              content_type='csv')

Training Shape: (6891, 83)


In [19]:
training_instance = 'ml.m5.2xlarge'
training_container = sagemaker.image_uris.retrieve(framework='tensorflow', region=my_region, version='2.3',
                                                   image_scope='training', instance_type=training_instance)

print("Success - the MySageMakerInstance is in the " + my_region + 
      " region. You will use the " + training_container + " container for your SageMaker endpoint.")

# Store model locally. A S3 URI would work too.
output_path = 's3://{}/{}/output/models/4/'.format(bucket_name, prefix)
source_dir = 'model/'
#training_instance = 'ml.m5.large'


tf_estimator = TensorFlow(entry_point='algo1.py',
                          role=role,
                          instance_count=1, 
                          instance_type=training_instance,   # Train on the local CPU ('local_gpu' if it has a GPU)
                          framework_version='2.2',  # try 2.2 "1.15
                          py_version='py37',
                          hyperparameters={'epochs': 40, 'batch_size': 32, 'model_name': '4'},
                          output_path=output_path, 
                          source_dir=source_dir, 
                          sagemaker_session=session#pipeline_session
                          # parameter 
                         )

# fit tensorflow estimator on training data
tf_estimator.fit({
    'train': 's3://{}/{}/train'.format(bucket_name, prefix),
    'test': 's3://{}/{}/val'.format(bucket_name, prefix),
    #'model_name': '/2'
})

# module_dir

Success - the MySageMakerInstance is in the us-east-1 region. You will use the 763104351884.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training:2.3-cpu-py37 container for your SageMaker endpoint.
2023-12-13 17:20:28 Starting - Starting the training job...
2023-12-13 17:20:53 Starting - Preparing the instances for trainingProfilerReport-1702488014: InProgress
.........
2023-12-13 17:22:13 Downloading - Downloading input data...
2023-12-13 17:22:54 Training - Training image download completed. Training in progress....2023-12-13 17:23:10.346346: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:425] Initializing the SageMaker Profiler.
2023-12-13 17:23:10.346489: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:106] SageMaker Profiler is not enabled. The timeline writer thread will not be started, future recorded events will be dropped.
2023-12-13 17:23:10.367878: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:425] Initializing the SageMaker Profiler.
2023-1